# Finding Anomolies

## Setup

### Import Data

In [1]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import warnings
import geopy.distance

warnings.filterwarnings('ignore')

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

storage = "/Volumes/easystore/Drones/"

flight_details = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/gpx-with-census-data.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

In [109]:
geocodio_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/geocodio/all-flights-manifest_geocodio.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

In [3]:
flight_details["geoid"] = flight_details["geoid"].astype(str)
flight_details["len"] = flight_details["geoid"].apply(lambda x: len(x))
flight_details.loc[flight_details["len"] == 14, "geoid"] = "0" + flight_details["geoid"]


In [93]:
geocodio_flights["date"] = pd.to_datetime(geocodio_flights["date"]).dt.date


### Launch Sites
- BayView Hospital = 60730131022000
- CVPD = 60730123021013
- SharpChula Medical Center = 60730133273000
- SWestern College = 60730134151000


## Functions

In [99]:
def compile_anomolies(anomolies):
    anomolies = anomolies[anomolies['id']!=False].copy()
    
    anomolies = pd.merge(anomolies,geocodio_flights,how='left',on='id')
    anomolies['fn'] = anomolies['id'].apply(lambda x: f"/Volumes/easystore/Drones/flights/kml/{x}.kml")

    compiled_flights = []
    for _, row in anomolies.iterrows():
        d = gpd.read_file(row['fn'], driver='KML')


        d['id'] = row['id']
        d['incident_type'] = row['type_y']
        d['incident_id'] = row['incident_id']
        d['address_map'] = row['address_map']
        d['date'] = row['date']
        d['time'] = row['time_s']
        d['lingering_block'] = row['lingering_block']
        d['seconds_lingering'] = row['seconds_lingering']
        d['destination_block'] = str(row['destination_block'])
        d['distance'] = str(row['distance'])
        d['latitude'] = row['Latitude']
        d['longitude'] = row['Longitude']
        d['lat_map'] = row['lat_map']
        d['lon_map'] = row['lon_map']
        d['starting_block'] = str(row['starting_block'])

        compiled_flights.append(d)
    return pd.concat(compiled_flights,ignore_index=True)

In [5]:
def get_unique_seconds_for_block(flight):

    times = pd.to_datetime(flight["sequence"])
    grouped_by_block_and_second = flight.groupby(
    ["id","geoid","block_group","tract", times.dt.hour, times.dt.minute, times.dt.second]).count()
    grouped_by_block_and_second.index.names = ["id","geoid","block_group","tract", "hour", "minute", "second"]
    grouped_by_block_and_second = grouped_by_block_and_second.reset_index()[
    ["id","geoid","block_group","tract", "hour", "minute", "second", "type"]
    ]
    grouped_by_block_and_second.columns = ["id","geoid","block_group","tract", "hour", "minute", "second", "count"]
    
    unique_seconds_in_block = grouped_by_block_and_second.groupby(["geoid","block_group","tract"]).count()
    unique_seconds_in_block = unique_seconds_in_block.reset_index()[["geoid","block_group","tract", "hour"]]
    unique_seconds_in_block.columns = ["GEOID20","block_group","tract","seconds"]
    return unique_seconds_in_block

## Searching

### Lingering over more than two NC blocks

In [5]:
def search_one(index):
#   search for flights that spent more than 5 minutes in two blocks that are not consecutive.
    LINGER = 300

    flight_id = flight_ids.iloc[index]
    flight = flight_details[flight_details['id']==flight_id]

    anomoly = {"id":False,"type":False}

    starting_block = flight.iloc[0]['geoid']
    
    unique_seconds_in_block = get_unique_seconds_for_block(flight)
    unique_seconds_in_block =  unique_seconds_in_block[unique_seconds_in_block['GEOID20']!=flight.iloc[0]['geoid']].copy()

    blocks_w_more_45 = unique_seconds_in_block[(unique_seconds_in_block['seconds']>LINGER)].copy()

    if blocks_w_more_45.shape[0] > 1:
        blocks_w_more_45['GEOID20'] = blocks_w_more_45['GEOID20'].astype(int)
        blocks_w_more_45 = blocks_w_more_45.sort_values("GEOID20")
        blocks_w_more_45['diff'] = blocks_w_more_45['GEOID20'].diff().abs().copy()
        result = blocks_w_more_45[(blocks_w_more_45['diff'] > 1)]
        if result.shape[0] > 0:
            anomoly = {"id":flight_id,"type":flight['type'].drop_duplicates().values[0]}
    return anomoly

#### Run Lingering NC Search

In [35]:
flight_ids = flight_details['id'].sample(50).drop_duplicates()
with Pool(2) as pool:

    lingering_nc = list(
        tqdm(pool.imap(search_one, range(0, flight_ids.shape[0])), total=flight_ids.shape[0])
    )
    lingering_nc = pd.DataFrame(lingering_nc)

  0%|          | 0/50 [00:00<?, ?it/s]

In [27]:
lingering_anomolies = compile_anomolies(lingering_nc)
lingering_anomolies = gpd.GeoDataFrame(lingering_anomolies)
lingering_anomolies['incident_type'] = lingering_anomolies['incident_type'].astype(str)

In [28]:
lingering_anomolies[['geometry','incident_type','id']].to_file("../data/lingering-nc.geojson", driver='GeoJSON')

In [29]:
lingering_anomolies.head()

,Name,Description,geometry,id,incident_type,incident_id,address_map,date,time
0,Home,,POINT Z (-117.08269 32.63997 22.13559),651d1096e03c7ba2e80d687d00e72ea0,DOMESTIC VIOLENCE,L062565,706 F,7-28-21,9:48am
1,Airdata.com,,"LINESTRING Z (-117.08269 32.63997 22.13559, -1...",651d1096e03c7ba2e80d687d00e72ea0,DOMESTIC VIOLENCE,L062565,706 F,7-28-21,9:48am
2,Home,,POINT Z (-117.07166 32.61747 34.84546),2d6bcf3670ab962c87609165668ddf7a,Fight,CVL40019,1242 Broadway,5-16-22,5:30pm
3,Airdata.com,,"LINESTRING Z (-117.07166 32.61747 34.84546, -1...",2d6bcf3670ab962c87609165668ddf7a,Fight,CVL40019,1242 Broadway,5-16-22,5:30pm


### Lingering over Block that is Not Destination
lingering for more than 5 minutes on a block thats in a diff tract than the stated destination AND that  tract is not the neighboring tract.

In [101]:
LINGER = 240
def search_two(index,details=False):
    if details == True:
        flight_id = index
        flight = flight_details[flight_details['id']==flight_id]
    else:
        flight_id = flight_ids.iloc[index]
        flight = flight_details[flight_details['id']==flight_id]

    manifest_details = geocodio_flights[geocodio_flights['id']==flight_id]
    destination_block = manifest_details.iloc[0][['Full FIPS (block)','Census Tract Code', 'Census Block Group']]
    
    destination_block['Census Tract Code'] = int(destination_block['Census Tract Code'])
    
    starting_block = flight.iloc[0]['geoid']
    anomoly = {"id":False,"type":False,'starting_block':starting_block,"destination_tract":destination_block['Census Tract Code']}

    unique_seconds_in_block = get_unique_seconds_for_block(flight)
    unique_seconds_in_block =  unique_seconds_in_block[unique_seconds_in_block['GEOID20']!=starting_block].copy()
    unique_seconds_in_block =  unique_seconds_in_block[unique_seconds_in_block['GEOID20']!=destination_block['Full FIPS (block)']].copy()

    lingering_blocks = unique_seconds_in_block[unique_seconds_in_block['seconds']>LINGER]
    
    
    if lingering_blocks.shape[0]>0:
        lingering_blocks_diff_tract = lingering_blocks[
            # (nd_blocks_g_5['block_group'].astype(int)!=destination_block['Census Block Group']) &
            (lingering_blocks['tract'].astype(int)!=int(destination_block['Census Tract Code']))
        ]
        if lingering_blocks_diff_tract.shape[0]>0:
            lingering_blocks_diff_tract['diff'] = lingering_blocks_diff_tract['tract'].astype(int).apply(lambda x: abs(x - int(destination_block['Census Tract Code']))).copy()
            lingering_blocks_non_consec_tract = lingering_blocks_diff_tract[lingering_blocks_diff_tract['diff']>1]
            if lingering_blocks_non_consec_tract.shape[0]>0:
                anomoly = {"id":flight_id,"type":flight['type'].drop_duplicates().values[0],'starting_block':starting_block,"destination_tract":destination_block['Census Tract Code']}

    if details == True:
        return [anomoly,unique_seconds_in_block,destination_block]
    else:
        return anomoly



In [110]:
# def calc_dist(flight_id):
#     flight = geocodio_flights[geocodio_flights['id']==flight_id]
#     dist = geopy.distance.geodesic((flight['lat_map'].values[0],flight['lon_map'].values[0]), (flight['Latitude'].values[0], flight['Longitude'].values[0])).miles
#     return dist

# geocodio_flights['map-delta'] = geocodio_flights['id'].apply(lambda x: calc_dist(x))

In [103]:
# geocodio_flights = geocodio_flights[geocodio_flights['map-delta']<=.01].copy()

In [257]:
# find flights where drones lingered over two blocks and one of those blocks was at least a mile from te map's stated destination.
# blacklist = [60730132062000,60730134092000]

LINGER = 240
def filter_one(index,details=False):
    
    if details == True:
        flight_id = index
        flight = flight_details[flight_details['id']==flight_id]
    else:
        flight_id = flight_ids.iloc[index]
        flight = flight_details[flight_details['id']==flight_id]
    try:
        manifest_details = geocodio_flights[geocodio_flights['id']==flight_id]
        destination_block = manifest_details.iloc[0][['Full FIPS (block)','Census Tract Code', 'Census Block Group','Latitude', 'Longitude']]
        destination_block['Census Tract Code'] = int(destination_block['Census Tract Code'])
        destination_block['Full FIPS (block)'] = int(destination_block['Full FIPS (block)'])
        starting_block = int(flight.iloc[0]['geoid'])

        unique_seconds_in_block = get_unique_seconds_for_block(flight)
        unique_seconds_in_block['GEOID20'].astype(int)
        
        unique_seconds_in_block =  unique_seconds_in_block[unique_seconds_in_block['GEOID20']!=starting_block].copy()
        unique_seconds_in_block =  unique_seconds_in_block[unique_seconds_in_block['GEOID20']!=destination_block['Full FIPS (block)']].copy()

        lingering_blocks = unique_seconds_in_block[unique_seconds_in_block['seconds']>LINGER]
        # lingering_blocks=lingering_blocks[~lingering_blocks['GEOID20'].isin(blacklist)]
        anomoly = {"id":False,"type":False,'starting_block':starting_block,"destination_tract":destination_block['Census Tract Code'],"lingering_block":False,"seconds_lingering":False}
#         consider increasing

        if lingering_blocks.shape[0]>0:
            lingering_blocks_far_from_dst = []
            for _,row in lingering_blocks.iterrows():
                lingering_lat,lingering_lng = flight[flight['geoid'] == row['GEOID20']][['latitude','longitude']].values[0]
                distance = geopy.distance.geodesic((lingering_lat,lingering_lng), (destination_block['Latitude'], destination_block['Longitude'])).miles
                if distance > 1:
                    anomoly ={"id":flight_id,"type":flight['type'].drop_duplicates().values[0],'starting_block':starting_block,"destination_block":int(destination_block['Full FIPS (block)']),"distance":distance,"lingering_block":row['GEOID20'],"seconds_lingering":row['seconds']}
        if details == True:
            return [anomoly,unique_seconds_in_block]
        else:
            return anomoly
    except:
        print("error")
        return {"id":False,"type":False,'starting_block':False,"destination_tract":False,"lingering_block":False,"seconds_lingering":False}
    


In [ ]:
flight_ids = flight_details[['id']].sample(frac=1).drop_duplicates()
flight_ids = flight_ids[flight_ids['id'].isin(geocodio_flights['id'])]['id']

with Pool(10) as pool:

    lingering_nd = list(
        tqdm(pool.imap(filter_one, range(0, flight_ids.shape[0])), total=flight_ids.shape[0])
    )
    lingering_nd = pd.DataFrame(lingering_nd)

In [ ]:
lingering_anomolies = compile_anomolies(lingering_nd)
lingering_anomolies = gpd.GeoDataFrame(lingering_anomolies)
lingering_anomolies['incident_type'] = lingering_anomolies['incident_type'].astype(str)
lingering_anomolies['destination_block']=lingering_anomolies['destination_block'].astype(float).astype(int)
lingering_anomolies['starting_block']=lingering_anomolies['starting_block'].astype(float).astype(int)
lingering_anomolies['lingering_block']=lingering_anomolies['lingering_block'].astype(float).astype(int)

lingering_anomolies.head()


In [ ]:
lingering_anomolies[['id','geometry','incident_type','distance','lingering_block','destination_block','longitude','latitude','address_map','starting_block']].to_file("../../data/lingering-nd.geojson", driver='GeoJSON')


In [279]:
# of the flights where a drone hovered for five minutes somewhere other than the stated destination...how many hovered in two blocksfor more than 5 min...and those two blocks are more than 1 mile apart
# blacklist = [60730132062000,60730134092000]
LINGER = 180
def filter_two(index,details=False):
    
    if details == True:
        flight_id = index
        flight = flight_details[flight_details['id']==flight_id]
    else:
        flight_id = flight_ids.iloc[index]
        flight = flight_details[flight_details['id']==flight_id]
    try:
        manifest_details = geocodio_flights[geocodio_flights['id']==flight_id]
        destination_block = manifest_details.iloc[0][['Full FIPS (block)','Census Tract Code', 'Census Block Group','Latitude', 'Longitude']]
        destination_block['Census Tract Code'] = int(destination_block['Census Tract Code'])
        destination_block['Full FIPS (block)'] = int(destination_block['Full FIPS (block)'])
        starting_block = int(flight.iloc[0]['geoid'])
        unique_seconds_in_block = get_unique_seconds_for_block(flight)
        unique_seconds_in_block['GEOID20'].astype(int)

        unique_seconds_in_block =  unique_seconds_in_block[unique_seconds_in_block['GEOID20']!=starting_block].copy()
        unique_seconds_in_block = unique_seconds_in_block[unique_seconds_in_block['GEOID20']!=destination_block['Full FIPS (block)']].copy()

        lingering_blocks = unique_seconds_in_block[unique_seconds_in_block['seconds']>LINGER]

        lingering_blocks =  lingering_blocks[lingering_blocks['GEOID20']!=starting_block].copy()
        lingering_blocks = lingering_blocks[lingering_blocks['GEOID20']!=destination_block['Full FIPS (block)']].copy()

        # lingering_blocks=lingering_blocks[~lingering_blocks['GEOID20'].isin(blacklist)]
        anomoly = {"id":False,"type":False,'starting_block':starting_block,"destination_tract":destination_block['Census Tract Code'],"lingering_block":False,"seconds_lingering":False}
    #         consider increasing

        if lingering_blocks.shape[0]>1:
            lingering_blocks_far_from_dst = []
            lingering_lat,lingering_lng = flight[flight['geoid'] == lingering_blocks.iloc[0]['GEOID20']][['latitude','longitude']].values[0]
            lingering_lat_2,lingering_lng_2 = flight[flight['geoid'] == lingering_blocks.iloc[-1]['GEOID20']][['latitude','longitude']].values[0]
            distance = geopy.distance.geodesic((lingering_lat,lingering_lng), (lingering_lat_2,lingering_lng_2)).miles
            if distance > 1:
                anomoly ={"id":flight_id,"type":flight['type'].drop_duplicates().values[0],'starting_block':starting_block,"destination_block":int(destination_block['Full FIPS (block)']),"distance":distance,"lingering_block":"Ignore","seconds_lingering":"Ignore"}
        if details == True:
            return [anomoly,unique_seconds_in_block]
        else:
            return anomoly
    except:
        print("error")
        return {"id":False,"type":False,'starting_block':False,"destination_tract":False,"lingering_block":False,"seconds_lingering":False}
    


In [272]:
lingering_anomolies = gpd.read_file('../../data/lingering-nd.geojson')


In [280]:
flight_ids = lingering_anomolies[['id']].drop_duplicates()['id']

with Pool(10) as pool:

    lingering_nd = list(
        tqdm(pool.imap(filter_two, range(0, flight_ids.shape[0])), total=flight_ids.shape[0])
    )
    lingering_nd = pd.DataFrame(lingering_nd)

  0%|          | 0/421 [00:00<?, ?it/s]

In [284]:
lingering_anomolies = compile_anomolies(lingering_nd)
lingering_anomolies = gpd.GeoDataFrame(lingering_anomolies)

lingering_anomolies.head()
lingering_anomolies[['id','geometry','incident_type','distance','lingering_block','destination_block','longitude','latitude','address_map','starting_block']].to_file("../../data/lingering-nd-filter-two.geojson", driver='GeoJSON')


In [283]:
lingering_anomolies

,Name,Description,geometry,id,incident_type,incident_id,address_map,date,time,lingering_block,seconds_lingering,destination_block,distance,latitude,longitude,lat_map,lon_map,starting_block
0,Home,,POINT Z (-116.99921 32.64016 148.54932),7c72b0202fdcc141be3af7e32887ec84,Vehicle Theft,CVL63664,Heritage Rd./ Santa Maya,7-31-21,2:02pm,Ignore,Ignore,60730133173001.0,1.980463451597837,32.597996,-117.004362,32.5983626,-117.0043805,60730134151000
1,Airdata.com,,"LINESTRING Z (-116.99921 32.64016 148.54932, -...",7c72b0202fdcc141be3af7e32887ec84,Vehicle Theft,CVL63664,Heritage Rd./ Santa Maya,7-31-21,2:02pm,Ignore,Ignore,60730133173001.0,1.980463451597837,32.597996,-117.004362,32.5983626,-117.0043805,60730134151000
2,Home,,POINT Z (-117.07165 32.61741 34.81735),b83aa6f08e90729be1ce8a8bc9d3db1b,Assault with Deadly Weapon,CVL15904,800 Palomar St./ 200 Palm Ave.,2-24-22,8:04am,Ignore,Ignore,60730131031000.0,2.3822413209270255,32.604507,-117.088406,32.60443,-117.088395,60730131022000
3,Airdata.com,,"LINESTRING Z (-117.07165 32.61741 34.81735, -1...",b83aa6f08e90729be1ce8a8bc9d3db1b,Assault with Deadly Weapon,CVL15904,800 Palomar St./ 200 Palm Ave.,2-24-22,8:04am,Ignore,Ignore,60730131031000.0,2.3822413209270255,32.604507,-117.088406,32.60443,-117.088395,60730131022000
4,Home,,POINT Z (-117.08272 32.63998 22.14278),0ccf51764ec53f5ea082b46d3aa03810,Assault / Reckless Driver,L076216 / L076218,400 Hilltop Dr / Broadway & F St,9-05-23,4:56pm,Ignore,Ignore,60730128001000.0,1.5553621101125665,32.640951,-117.066502,32.6412731,-117.0663321,60730123021013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Airdata.com,,"LINESTRING Z (-117.08273 32.64002 22.15444, -1...",e9c344c5d6811cb50582b0bb22ca892f,Traffic Collision and Fire,CVL051730 and CVL051732,"805 and E ST, 400 Oaklawn",6-19-23,2:32pm,Ignore,Ignore,60730124021002.0,1.7365952512535299,32.644111,-117.084006,32.6439658,-117.0840642,60730123021013
92,Home,,POINT Z (-117.02300 32.61980 139.68118),589a322768b81fd4db9347f95532ce53,Alarm Call/ Suspicious Circumstance,L058519/L058516,500 Telegraph Canyon Rd/ 500 Paseo Burga,7-11-23,1:35pm,Ignore,Ignore,60730134212005.0,1.1209797952042044,32.647238,-117.024319,32.6474275,-117.0243611,60730133273000
93,Airdata.com,,"LINESTRING Z (-117.02300 32.61980 139.68118, -...",589a322768b81fd4db9347f95532ce53,Alarm Call/ Suspicious Circumstance,L058519/L058516,500 Telegraph Canyon Rd/ 500 Paseo Burga,7-11-23,1:35pm,Ignore,Ignore,60730134212005.0,1.1209797952042044,32.647238,-117.024319,32.6474275,-117.0243611,60730133273000
94,Home,,POINT Z (-117.02297 32.61981 139.85410),2b69e41f2ff196f75991e286ba11edde,Suspicious person,CVL052532,800 Floyd Ave,6-28-22,2:10pm,Ignore,Ignore,60730134091008.0,1.3627714285705121,32.633563,-117.04204,32.6336546,-117.0421048,60730133273000


ValueError: No objects to concatenate

In [ ]:
# lingering_anomolies = lingering_anomolies[lingering_anomolies['distance'].astype(float)<=5]

In [ ]:
lingering_anomolies[['id','geometry','incident_type','distance','lingering_block','destination_block','longitude','latitude','address_map','starting_block']].to_file("../../data/lingering-nd.geojson", driver='GeoJSON')


In [ ]:
lingering_anomolies.shape[0]

In [201]:
grouped = lingering_anomolies.groupby('lingering_block').count().reset_index()[['lingering_block','Name']]

In [203]:
grouped.sort_values('Name').tail(20)

,lingering_block,Name
66,60730125022000,8
6,60070016021023,8
8,60070016021026,8
70,60730125023001,8
78,60730126001010,8
88,60730127002000,10
179,60730133173045,10
45,60730124012001,10
128,60730131032011,10
62,60730125021006,10


In [204]:
a = lingering_anomolies[lingering_anomolies['lingering_block']==60730123031006]

a[['id','geometry','incident_type','distance','lingering_block','destination_block','longitude','latitude','address_map','starting_block']].to_file("../../data/block-lingering-nd.geojson", driver='GeoJSON')
a

,Name,Description,geometry,id,incident_type,incident_id,address_map,date,time,lingering_block,seconds_lingering,destination_block,distance,latitude,longitude,lat_map,lon_map,starting_block
78,Home,,POINT Z (-117.08271 32.63997 22.14589),beccadaaaf63d4ecb250d2295fbc2a3f,Fire,CVL21298,I-805/ Bonita Rd.,3-14-22,4:10pm,60730123031006,794,60730133091001,2.3343170757688103,32.621982,-117.040562,32.6220554,-117.0405308,60730123021013
79,Airdata.com,,"LINESTRING Z (-117.08271 32.63997 22.14589, -1...",beccadaaaf63d4ecb250d2295fbc2a3f,Fire,CVL21298,I-805/ Bonita Rd.,3-14-22,4:10pm,60730123031006,794,60730133091001,2.3343170757688103,32.621982,-117.040562,32.6220554,-117.0405308,60730123021013
126,Home,,POINT Z (-117.08271 32.63996 22.16603),c61d95e48edacd51b395ea9ebfcd2a08,Collision,CVL51924,I-805/ Bonita Rd.,6-26-22,12:16pm,60730123031006,602,60730133091001,2.3583472674222925,32.621982,-117.040562,32.6220554,-117.0405308,60730123021013
127,Airdata.com,,"LINESTRING Z (-117.08271 32.63996 22.16603, -1...",c61d95e48edacd51b395ea9ebfcd2a08,Collision,CVL51924,I-805/ Bonita Rd.,6-26-22,12:16pm,60730123031006,602,60730133091001,2.3583472674222925,32.621982,-117.040562,32.6220554,-117.0405308,60730123021013
138,Home,,POINT Z (-117.08270 32.63998 22.13710),856f7b069c1f0e4f53e690d7e956c61d,Fire,CVL77703,I-805/ Bonita Rd.,9-18-22,1:46pm,60730123031006,464,60730133091001,2.3343170757688103,32.621982,-117.040562,32.6220554,-117.0405308,60730123021013
139,Airdata.com,,"LINESTRING Z (-117.08270 32.63998 22.13710, -1...",856f7b069c1f0e4f53e690d7e956c61d,Fire,CVL77703,I-805/ Bonita Rd.,9-18-22,1:46pm,60730123031006,464,60730133091001,2.3343170757688103,32.621982,-117.040562,32.6220554,-117.0405308,60730123021013
518,Home,,POINT Z (-117.08271 32.63996 22.16603),50299bf05e7e0564236734c025c8712a,Robbery,CV2200308,700 E,1-08-22,6:57am,60730123031006,1121,60730125022001,1.4145037909389961,32.639895,-117.097631,32.639889,-117.0976342,60730123021013
519,Airdata.com,,"LINESTRING Z (-117.08271 32.63996 22.16603, -1...",50299bf05e7e0564236734c025c8712a,Robbery,CV2200308,700 E,1-08-22,6:57am,60730123031006,1121,60730125022001,1.4145037909389961,32.639895,-117.097631,32.639889,-117.0976342,60730123021013
538,Home,,POINT Z (-117.08271 32.63998 22.13929),9ed19e6496e8097821f1f6b6cefe4b0d,Fire,CVL23304,00 First Ave.,3-21-22,4:35pm,60730123031006,1256,60730129001011,1.6420116005435215,32.626136,-117.065163,32.626324,-117.064987,60730123021013
539,Airdata.com,,"LINESTRING Z (-117.08271 32.63998 22.13929, -1...",9ed19e6496e8097821f1f6b6cefe4b0d,Fire,CVL23304,00 First Ave.,3-21-22,4:35pm,60730123031006,1256,60730129001011,1.6420116005435215,32.626136,-117.065163,32.626324,-117.064987,60730123021013
